In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import numpy as np
#%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import sparse
import pandas as pd

# Dummy Model

In [ ]:
class Dummy:
  def _init_(self):
    pass
  
  def fit(self, x_train, y_train):
    self.x_train = x_train
    self.y_train = y_train
    labels, counts = np.unique(y_train, return_counts=True)
    self.pred_class = np.argmax(counts)
    return self.pred_class
  
  def predict(self, x_test):
    return [self.pred_class]*x_test.shape[0]

# Data

In [3]:
x_train = np.genfromtxt("/content/drive/My Drive/IFT_FML/comp1/data/train.csv", delimiter=',', skip_header=1)
x_train = x_train[:, :-1]
labels = np.genfromtxt('/content/drive/My Drive/IFT_FML/comp1/data/train_result.csv', delimiter=',', skip_header=1)
labels = labels[:, 1]
x_test = np.genfromtxt('/content/drive/My Drive/IFT_FML/comp1/data/test.csv', delimiter=',', skip_header=1)
x_test = x_test[:, :-1]


In [ ]:
print(x_test.shape[0])

x_train[:10, :]

10000


array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -7.8071055e-24,  2.3793220e-24, -5.5820960e-26],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -5.8677304e-23,  4.8580465e-24, -4.5954980e-25],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -3.5995640e-23,  7.4711936e-24, -3.8152997e-24],
       ...,
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         1.5096557e-25,  1.3522884e-24,  8.5045970e-25],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         4.5009447e-23,  2.0409717e-23,  9.0093030e-24],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         1.2156436e-23,  1.1229318e-24,  3.0903960e-25]])

In [ ]:
labels[:10]

array([ 5.,  4.,  5.,  4.,  4., 12., 16., 14.,  7., 10.])

50000

# Running Dummy Model

In [ ]:
mod = Dummy()
mod.fit(x_train, labels)

9

In [1]:
y_pred = mod.predict(x_test)

NameError: ignored

# Logistic Regression Model

In [6]:
from tqdm import tqdm

logistic = lambda z: 1./ (1 + np.exp(-z))
class LogisticRegression:  
  """
  M: Number of classes
  D: Feature Dimension
  N: Number of rows
  """
  def __init__(self, learning_rate=.001, max_iters=1e4, eps=1e-8):
    self.learning_rate = learning_rate
    self.max_iters = max_iters
    self.epsilon = eps
    

  def gradient(self, x, y, params):
    # assumes x has a bias term (ones)
    N = x.shape[0] # N number of rows
    w = params

    yh = logistic(np.dot(x, w))
    dy = yh - y
    dw = np.dot(x.T, dy)/N 
    dparams = dw
    return dparams

  def error(self, x, y, params):
    y_probs = logistic(np.matmul(x, params))
    yh = np.argmax(y_probs, axis=1)
    return (yh != y).sum()/x.shape[0]

  def fit(self, x, y, k=10):
    N = x.shape[0]
    x = np.c_[ x, np.ones(N) ]
    D = x.shape[1]

    self.M = len(np.unique(y)) # Number of classes

    train_idx, valid_idx = [], []

    # Create Validation and Train sets
    indices = np.arange(x.shape[0])
    splits = np.array_split(indices, k)

    valid_idx = splits[0].tolist()
    temp_train = []
    for i in range(1, len(splits)-1):
      temp_train += splits[i].tolist()

    x_train, y_train = x[temp_train, :], y[temp_train]
    x_valid, y_valid = x[valid_idx, :], y[valid_idx]

    # Encode Y
    y_train_enc = pd.get_dummies(y_train)

    # Init parameters
    params = np.random.randn(D, self.M) * .01
    self.params = params

    # Gradient Descent
    grads_normed = np.array([np.inf]*3)
    self.valid_err = [np.inf]
    # print(self.error(x_valid, y_valid, params))
    t = 1
    pbar = tqdm(total=self.max_iters)
    while (np.any(grads_normed > self.epsilon) and t < self.max_iters
            or self.valid_err[-1] > (self.error(x_valid, y_valid, params)+.001)):
        self.params = params
        t += 1
        self.valid_err.append(self.error(x_valid, y_valid, params))

        grad = self.gradient(x_train, y_train_enc, params)
        params -= self.learning_rate * grad

        grads_normed = np.array([np.linalg.norm(g) for g in grad])
        pbar.update(1)
    pbar.close()
    print(self.valid_err)
    # self.params = self.grad_descent(self.gradient, x_train, y_train_enc, w)


  def predict(self, x):
      w = self.params
      x = np.c_[ x, np.ones(x.shape[0]) ]
      y_probs = logistic(np.dot(x, w)) 
      yh = np.argmax(y_probs, axis=1)
      yh = np.c_[np.arange(0, yh.shape[0], 1, dtype=int), yh]
      return yh

# Running Logistic Regression

In [9]:
x_train.shape

(50000, 1568)

In [4]:
np.std(x_train, axis=0).shape

(1568,)

In [ ]:
x_train_log = x_train-
x_test_log = 

In [7]:
log_mod = LogisticRegression()
log_mod.fit(x_train, labels)


100%|█████████▉| 9999/10000.0 [1:48:18<00:00,  1.54it/s]

[inf, 0.9454, 0.9456, 0.945, 0.9438, 0.944, 0.9428, 0.9416, 0.9402, 0.9394, 0.9378, 0.9372, 0.9356, 0.9362, 0.9358, 0.935, 0.9348, 0.9344, 0.934, 0.933, 0.933, 0.9326, 0.931, 0.931, 0.9304, 0.9298, 0.93, 0.929, 0.928, 0.9268, 0.9268, 0.9266, 0.9274, 0.9268, 0.9266, 0.927, 0.9266, 0.9266, 0.9264, 0.9258, 0.9254, 0.9258, 0.9262, 0.9256, 0.9252, 0.9254, 0.9244, 0.924, 0.9248, 0.9238, 0.9242, 0.9242, 0.9238, 0.9238, 0.9244, 0.9248, 0.9244, 0.9234, 0.9232, 0.9228, 0.9224, 0.9228, 0.9232, 0.923, 0.9222, 0.9222, 0.9228, 0.9226, 0.9224, 0.9222, 0.922, 0.9218, 0.921, 0.9204, 0.9202, 0.92, 0.9198, 0.9192, 0.9192, 0.9184, 0.9182, 0.9182, 0.9188, 0.919, 0.9184, 0.9186, 0.9186, 0.919, 0.919, 0.9194, 0.919, 0.919, 0.9186, 0.9182, 0.9184, 0.9182, 0.9186, 0.9184, 0.9182, 0.9186, 0.9184, 0.9188, 0.919, 0.9184, 0.918, 0.9178, 0.9178, 0.9176, 0.917, 0.9166, 0.9162, 0.9158, 0.916, 0.9154, 0.9158, 0.9158, 0.9158, 0.916, 0.9158, 0.915, 0.9146, 0.9146, 0.9144, 0.9142, 0.9144, 0.9148, 0.9144, 0.9142, 0.9138, 

In [ ]:
yh_test = log_mod.predict(x_test_log)

In [ ]:
np.savetxt("/content/drive/My Drive/IFT_FML/comp1/data/test_res.csv", yh_test, delimiter=",", header="Index,Class", fmt="%i", comments='')

# CNN

In [ ]:
def split_train_valid(x, y, k=5):
  # Create Validation and Train sets
    indices = np.arange(x.shape[0])
    splits = np.array_split(indices, k)

    valid_idx = splits[0].tolist()
    temp_train = []
    for i in range(1, len(splits)-1):
      temp_train += splits[i].tolist()
    
    x_train = x[temp_train, :] 
    y_train = y.iloc[temp_train, :]
    x_valid, y_valid = x[valid_idx, :], y.iloc[valid_idx, :]

    return x_train, y_train, x_valid, y_valid

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import SGD
# metrics 
from keras.metrics import categorical_crossentropy
# optimization method
from keras.optimizers import SGD

def LeNet():
  model = Sequential()
  
  # Convolutional layer  
  model.add(Conv2D(filters = 6, kernel_size = (5,5), padding = 'same', 
                   activation = 'relu', input_shape = (28,56,1)))
  
  # Max-pooing layer with pooling window size is 2x2
  model.add(MaxPooling2D(pool_size = (2,2)))
  
  # Convolutional layer 
  model.add(Conv2D(filters = 16, kernel_size = (5,5), activation = 'relu'))
  
  # Max-pooling layer 
  model.add(MaxPooling2D(pool_size = (2,2)))
  
  # Flatten layer 
  model.add(Flatten())
  
  # The first fully connected layer 
  model.add(Dense(120, activation = 'relu'))
  
  # The output layer  
  model.add(Dense(19, activation = 'softmax'))
  
  # compile the model with a loss function, a metric and an optimizer function
  # In this case, the loss function is categorical crossentropy, 
  # we use Stochastic Gradient Descent (SGD) method with learning rate lr = 0.01 to optimize the loss function
  # metric: accuracy 
  
  opt = SGD(lr = 0.01)
  model.compile(loss = categorical_crossentropy, 
                optimizer = opt, 
                metrics = ['accuracy']) 
                
  return model

In [ ]:
def train_lenet(model, x, y, batch_size=128, epochs = 50):
    '''
    x (_, 28, 28, 1)
    y one hot encoded : (_, 10)

    '''

    # Split Train/Validation
    x_train, y_train, x_valid, y_valid = split_train_valid(x, y)
    # batch_size = 128
    # print(y_valid0.shape)
    # print(y_train0.shape)

    acc = model.fit(x_train, y_train, epochs = epochs,       # HYPERPARAMETER 
                            batch_size = batch_size,             # HYPERPARAMETER
                          steps_per_epoch = x_train.shape[0]//batch_size, 
                          validation_data = (x_valid, y_valid), 
                          validation_steps = x_valid.shape[0]//batch_size, verbose = 1)
    return None

def predict(model, x):
  '''
  input:
  - x: (_, 28, 28, 1)
  output:
  - y: (_, 1)
  '''
  vec_p = model.predict(x)
  # determine the label corresponding to vector vec_p
  y_p = np.argmax(vec_p, axis=1)
  return y_p

# Run LeNet

In [ ]:
# Reshape for Keras
x_train_tens, x_train_ones = x_train[:, :784].reshape(x_train.shape[0], 28, 28, 1), x_train[:, 784:].reshape(x_train.shape[0], 28, 28, 1)
y_train_tens, y_train_ones = labels//10, labels % 10
x_test_tens, x_test_ones = x_test[:, :784].reshape(x_test.shape[0], 28, 28, 1), x_test[:, 784:].reshape(x_test.shape[0], 28, 28, 1)

zeros = pd.DataFrame(np.zeros((y_train_tens.shape[0], 8)))
y_train_tens, y_train_ones = pd.concat([pd.get_dummies(y_train_tens), zeros],axis=1), pd.get_dummies(y_train_ones)

# x_train_tens, y_train_tens, x_valid_tens, y_valid_tens = split_train_valid(x_train_tens, y_train_tens)

(50000,)
(50000, 10)
(50000, 10)


In [ ]:
mod_tens = LeNet()
train_lenet(mod_tens, x_train_tens, y_train_tens, epochs=100)
y_tens = predict(mod_tens, x_test_tens)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/4
234/234 [==============================] - 28s 115ms/step - loss: 0.7075 - accuracy: 0.6391 - val_loss: 0.5897 - val_accuracy: 0.6928
Epoch 2/4
234/234 [==============================] - 23s 99ms/step - loss: 0.5789 - accuracy: 0.6964 - val_loss: 0.5630 - val_accuracy: 0.7134
Epoch 3/4
234/234 [==============================] - 20s 87ms/step - loss: 0.5417 - accuracy: 0.7298 - val_loss: 0.5277 - val_accuracy: 0.7363
Epoch 4/4
234/234 [==============================] - 22s 93ms/step - loss: 0.5153 - accuracy: 0.7468 - val_loss: 0.5019 - val_accuracy: 0.7530


In [ ]:
mod_ones = LeNet()
train_lenet(mod_ones, x_train_ones, y_train_ones, epochs=4)
y_ones = predict(mod_ones, x_test_ones)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/4
234/234 [==============================] - 21s 86ms/step - loss: 2.3047 - accuracy: 0.1055 - val_loss: 2.3000 - val_accuracy: 0.1086
Epoch 2/4
234/234 [==============================] - 21s 90ms/step - loss: 2.2958 - accuracy: 0.1211 - val_loss: 2.2946 - val_accuracy: 0.1221
Epoch 3/4
234/234 [==============================] - 23s 100ms/step - loss: 2.2895 - accuracy: 0.1335 - val_loss: 2.2876 - val_accuracy: 0.1394
Epoch 4/4
234/234 [==============================] - 22s 95ms/step - loss: 2.2818 - accuracy: 0.1436 - val_loss: 2.2797 - val_accuracy: 0.1522


In [ ]:
# Combine y_ones y_tens
y_pred = np.c_[np.arange(y_tens.shape[0]), y_tens*10+y_ones]

In [ ]:
np.savetxt("/content/drive/My Drive/IFT_FML/comp1/data/submission.csv", y_pred, delimiter=",", header="Index,Class", fmt="%i", comments='')

array([   0,    1,    2, ..., 9997, 9998, 9999])

# Plots

In [ ]:
np.arange()

In [ ]:
import matplotlib.pyplot as plt
def plot_valid_acc(acc:list):
  plt.plot(np.arange(len(acc)),acc)